In [1]:
import pandas as pd
import numpy as np
import lightgbm
import catboost
import tensorflow as tf
import lightgbm
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor

In [2]:
train1 = pd.read_csv("datasets/median_data_train.csv")
test1 = pd.read_csv("datasets/median_data_test.csv")

In [3]:
test1_dates = test1["tarih"]
train1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
test1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)

In [4]:
test1.head()

,ilce,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,0,38.7996,26.9707,14.15,24.85,0.0,83.10,167.85,4.25,1.0,14.50
1,izmir-aliaga,0,38.7996,26.9707,15.20,9.05,0.0,79.85,159.10,1.65,1.0,15.30
2,izmir-aliaga,0,38.7996,26.9707,14.85,52.05,0.0,69.75,133.75,4.00,1.0,14.50
3,izmir-aliaga,0,38.7996,26.9707,14.95,81.60,0.0,78.45,175.60,8.05,1.0,14.35
4,izmir-aliaga,0,38.7996,26.9707,14.85,52.35,0.0,68.45,187.80,6.90,1.0,13.60


In [5]:
train1.head()

,ilce,bildirimsiz_sum,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,5.0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.0,13.00
1,izmir-aliaga,3.0,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.0,12.60
2,izmir-aliaga,2.0,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.0,12.00
3,izmir-aliaga,0.0,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.0,14.05
4,izmir-aliaga,8.0,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.0,14.20


In [6]:
ord_coder = OrdinalEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

In [7]:
X = train1.drop("bildirimsiz_sum",axis=1)
y = train1["bildirimsiz_sum"]

In [8]:
dummies = ohe.fit_transform(X[["ilce"]])
dummies = dummies.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummy = pd.DataFrame(data=dummies,columns=features)

In [9]:
X = X.drop("ilce",axis=1)

In [10]:
X = pd.concat([X,dummy],axis=1)

In [11]:
X

,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,...,ilce_manisa-koprubasi,ilce_manisa-kula,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre
0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51601,0.0,39.1881,27.6100,9.75,70.15,0.0,86.80,225.30,1.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51602,1.0,39.1881,27.6100,11.75,24.60,0.0,74.65,153.05,2.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51603,0.0,39.1881,27.6100,12.50,70.05,0.0,71.65,152.70,3.80,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51604,1.0,39.1881,27.6100,14.00,66.35,0.0,76.75,155.70,2.90,1.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
X["ilce"] = ord_coder.fit_transform(X[["ilce"]])

In [11]:
minmax = MinMaxScaler()

In [12]:
cols = dummy.columns.to_list()

In [13]:
cols.append("bildirimli_sum")

In [14]:
mm_columns = X.drop(cols,axis=1).columns

In [24]:
mm_columns = X.drop(["ilce","bildirimli_sum"],axis=1).columns

In [15]:
minmax.fit(X[mm_columns])
X[mm_columns] = minmax.fit_transform(X[mm_columns])

In [16]:
X

,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,...,ilce_manisa-koprubasi,ilce_manisa-kula,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre
0,0.0,0.68601,0.260122,0.484581,0.5075,0.0,0.858989,0.473528,0.309028,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.68601,0.260122,0.475771,0.3640,0.0,0.810811,0.295214,0.187500,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.68601,0.260122,0.468062,0.5730,0.0,0.752056,0.334604,0.218750,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.68601,0.260122,0.513216,0.4805,0.0,0.622209,0.315262,0.295139,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.68601,0.260122,0.514317,0.2265,0.0,0.755582,0.498518,0.253472,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51601,0.0,1.00000,0.509255,0.416300,0.7015,0.0,0.844888,0.629677,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51602,1.0,1.00000,0.509255,0.460352,0.2460,0.0,0.702115,0.425667,0.152778,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51603,0.0,1.00000,0.509255,0.476872,0.7005,0.0,0.666863,0.424679,0.243056,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51604,1.0,1.00000,0.509255,0.509912,0.6635,0.0,0.726792,0.433150,0.180556,0.010235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
dummies = ohe.transform(test1[["ilce"]])
dummies = dummies.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummy = pd.DataFrame(data=dummies,columns=features)
test1 = test1.drop("ilce",axis=1)
test1 = pd.concat([test1,dummy],axis=1)
cols = dummy.columns.to_list()
cols.append("bildirimli_sum")
mm_columns = test1.drop(cols,axis=1).columns
test1[mm_columns] = minmax.transform(test1[mm_columns])

In [27]:
test1["ilce"] = ord_coder.transform(test1[["ilce"]])
mm_columns = test1.drop(["ilce","bildirimli_sum"],axis=1).columns
test1[mm_columns] = minmax.transform(test1[mm_columns])

In [22]:
regressor = catboost.CatBoostRegressor(
    n_estimators=400,
    learning_rate=0.05,
    depth=9,
    l2_leaf_reg=3
)
regressor = SVR(kernel="poly",degree=2,C=1.5)

In [23]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
maes = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    maes.append(mean_absolute_error(y_test,y_pred))

c:\Users\LOKMAN\anaconda3\envs\MlEnv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [25]:
maes

[2.743083244493639,
 2.722544339421059,
 2.7347068263110303,
 2.7332868386463165,
 2.7281488320428986]

In [26]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
predictions = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(test1)
    predictions.append(y_pred)

c:\Users\LOKMAN\anaconda3\envs\MlEnv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [27]:
predictions

[array([4.51040772, 4.17171347, 4.65655165, ..., 6.45999531, 6.52521285,
        7.20480194]),
 array([4.67408917, 4.35888402, 4.86283417, ..., 6.98154685, 7.10786503,
        7.27651279]),
 array([4.45307929, 4.15930436, 4.57650846, ..., 6.77823724, 6.80855459,
        7.28815643]),
 array([4.57148642, 4.25805365, 4.71345383, ..., 6.83974471, 6.75242456,
        7.24401194]),
 array([4.55139311, 4.24095396, 4.65573119, ..., 6.84961366, 6.8676025 ,
        7.24737625])]

In [28]:
predicteds = np.mean(predictions,axis=0)

In [29]:
predicteds

array([4.55209114, 4.23778189, 4.69301586, ..., 6.78182755, 6.81233191,
       7.25217187])

In [31]:
inverse_ilceler = ohe.inverse_transform(test1[dummy.columns])

In [32]:
test1 = test1.drop(dummy.columns,axis=1)

In [33]:
inverse_ilceler = inverse_ilceler.reshape((1316,))

In [34]:
test1["ilce"] = inverse_ilceler
test1["tarih"] = test1_dates

In [35]:
test1["tarih"] = test1["tarih"].astype("str")

In [36]:
unique_id = test1["tarih"] + "-" + test1["ilce"]

In [37]:
submission = pd.DataFrame({"unique_id":unique_id,"bildirimsiz_sum":predicteds})

In [107]:
submission.to_csv("submission2.csv",index=False)

In [38]:
submission.to_csv("submission_SVR.csv",index=False)

In [39]:
import joblib
joblib.dump(regressor,"regressor_svr.joblib")

['regressor_svr.joblib']